In [47]:
using Gurobi, Cbc, Mosek, PyPlot, JuMP, Ipopt

## Tyre specs
coflo = 0.2 #coeffecient of friction longitudnal(in direction of motion)
cofla = 0.4 #coeffecient of friction lattitudnal(perpendicular to direction of motion of the front tyers when they turn)


## car specs all specs in meters and seconds
# mass = 1000 # in kg
g = 9.8  # gravitational pull in m/sec^2
maxacc = 6.5 # max acc in m/sec^2
maxv = 55.55 # max velocity in m/sec equivalent to 200km/hr
maxTu = 1.27 #max steer one side


##model
m = Model(solver = IpoptSolver(print_level=0))
    
    
## Start ad Finish constraints
k =10
startPos = [50,0]
finishPos = [50, 100]
# finishPos = [25000,180000]
startVel = [0,10]
finishVel = [0,0]
finishAcc = [0,0]

## Basic variables and their constraints
@variable(m, P[1:2,1:k])  # resulting position in x,y
@variable(m, v[1:2,1:k])
@variable(m, -(2*maxacc) <= Thrust[1:k] <= maxacc)  # thruster input
# @variable(m, 0 <= Braking[1:k] <= 2*maxacc)  # braking input
@variable(m, AccDM[1:2,1:k])
@variable(m, AccDMBeta[1:k] >= 0)
@variable(m, AccCent[1:2,1:k]) 

#  @variable(m, z[1:k], Bin)
# @constraint(m,(((AccDM[1,:].^2) + (AccDM[2,:].^2)) + ((v[1,:].^2) + (v[2,:].^2))) .<= (maxacc^2 + maxv^2)* z - 1(1 - z))                    # if LHS>0 then z=1
# @constraint(m,  (AccCent[1,:].^2 + AccCent[2,:].^2) .>= -((cofla*g)^2) * (1 - z))


@constraint(m, P[:,1] .== startPos)
@constraint(m, v[:,1] .== startVel)
# @constraint(m, v[:,k] .== finishVel)
# @constraint(m, AccDM[:,k] .== finishAcc)
@expression(m, finishDistance, sum((P[i,k]-finishPos[i])^2 for i in 1:2))


## motion constraints
for t in 1:k
    @constraint(m, AccDM[1,t]*AccCent[1,t] + AccDM[2,t]*AccCent[2,t] == 0)
    @constraint(m, ((AccDM[1,t]^2)+(AccDM[2,t]^2)) == -(Thrust[t])^2)
    @constraint(m, (AccCent[1,t]^2)+(AccCent[2,t]^2) <= (cofla*g)^2)
    @constraint(m, (v[1,t]^2)+(v[2,t]^2) <= maxv^2)
end

for t in 1:k-1
    @constraint(m, AccDM[:,t] .== AccDMBeta[t]*(P[:,t+1]-P[:,t]))
    @constraint(m, v[1,t+1] == v[1,t] + AccDM[1,t] + AccCent[1,t])
    @constraint(m, v[2,t+1] == v[2,t] + AccDM[2,t] + AccCent[2,t])  
    @constraint(m, P[1,t+1] == P[1,t] + v[1,t] + (.5)*(AccDM[1,t] + AccCent[1,t])) 
    @constraint(m, P[2,t+1] == P[2,t] + v[2,t] + (.5)*(AccDM[2,t] + AccCent[2,t]))     
end


# for t in 2:k-2
#     @NLconstraint(m, (atan(v[2,t]/v[1,t])-atan(v[2,t+1]/v[1,t+1])) <= maxTu)
#     @NLconstraint(m, (atan(v[2,t]/v[1,t])-atan(v[2,t+1]/v[1,t+1])) >= -maxTu)
# end


## Track boundaries variables and constraints
@constraint(m, 0 .<= P[1,:] .<= 5000)
for t in 1:k
    @constraint(m, P[2,t] <= 5000 + sqrt(2500^2 - P[1,t]^2))
    @constraint(m, (P[1,t]-2500)^2 + (P[2,t]-5000)^2 >= 2400^2)
end


## Objective function
@objective(m, Min, finishDistance)
solve(m)

rad = linspace(0,π,100) 

x = 2500 + 2400*cos.(rad)
y = 5000 +2400*sin.(rad)
x1 = 2500 + 2500*cos.(rad)
y1 = 5000 + 2500*sin.(rad)

x2 = linespace(0,200,100)
x3 = linespace(100,198.66,100)
x4 = linespace(200,400,100)
x5 = linespace(198.66,401.33,100)
x6 = linespace(400,600,100)
x7 = linespace(401.33,500,100)

y2 = (3/2)*x2 + 2000



Llinex = [0, 0]
Lliney = [0, 5000]
LMlinex = [100,100]
LMliney = [0,5000]
RMlinex = [4900, 4900]
RMliney = [0,5000]
Rlinex = [5000,5000]
Rliney = [0,5000]

figure(figsize=(12,12))
title("Position")

plot(Llinex[:],Lliney[:], color = "#000000", markersize=4 )
plot(LMlinex[:],LMliney[:],color = "#000000", markersize=4 )
plot(RMlinex[:],RMliney[:], color = "#000000", markersize=4 )
plot(Rlinex[:],Rliney[:], color = "#000000", markersize=4 )
plot(x,y, color = "#000000", markersize=4 )
plot(x1,y1, color = "#000000", markersize=4 )
plot(x2,y2, color = "#000000", markersize=4 )

plot( getvalue(P[1,:]), getvalue(P[2,:]), "b.-", markersize=4 )

# axis((-20000,80000,-10000,200000));
# axis((-20,80,-10,200));
# # axis("equal")
# figure(figsize=(12,4))
# title("velocity x")
# plot( getvalue(v[1,:]), "b.", markersize=12 )
# figure(figsize=(12,4))
# title("velocity y")
# plot( getvalue(v[2,:]), "b-", markersize=12 )
# figure(figsize=(12,4))
# title("Acceleration in direction of motion")
# plot( getvalue(Acc[1,:]), "b-", markersize=12 )
# figure(figsize=(12,4))
# title("Centripital acceleration")
# plot( getvalue(Acc[2,:]), "b-", markersize=12 )
# figure(figsize=(12,4))
# title("Thrust")
# plot( getvalue(Thrust), "b-", markersize=12 )
# figure(figsize=(12,4))
# title("Braking")
# plot( getvalue(Braking), "b-", markersize=12 )

LoadError: [91msqrt is not defined for type QuadExpr. Are you trying to build a nonlinear problem? Make sure you use @NLconstraint/@NLobjective.[39m